In [2]:
print('a')

a


In [1]:
'''
Convert synthesize data to 
entity2id.txt, relation2id, entity2id_objType.txt
'''

import json
import pickle
import os
from tqdm.notebook import tqdm
from typing import Set, Tuple, Generator
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


OUTPUT_PATH = './data/3_openKE/synthesize'
INPUT_PATH = '/workdir/home/bai/data_processing/dgl/data_new/DARPA/before_embedding/3.10/synthesized_format.json'

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)

In [2]:
data = list()
with open(INPUT_PATH, 'r') as f:
    for line in tqdm(f):
        event = json.loads(line)
        data.append(event)
        

# # Process each line as a separate JSON object
# for line in tqdm(data, desc='Loading JSON'):
#     try:
#         item = json.loads(line)  # Parse the JSON object
#         if item.get('label') == 'benign':
#             print(item)
#             break
            
#     except json.JSONDecodeError as e:
#         print(f"Error decoding JSON: {e}")

0it [00:00, ?it/s]

In [5]:
print(len(data))
type(data[1])

42899811


dict

In [6]:
data[1]

{'src_uuid': '966D983C-607E-4039-BEEF-5E76BC0A4DA8',
 'src_node': {'Cmdline': 'None', 'Type': 'process', 'Pid': 9432},
 'dst_uuid': '1987EE68-E918-4765-8082-3DE89890A305',
 'dst_node': {'Filepath': '\\Device\\HarddiskVolume2\\Users\\admin\\AppData\\Roaming\\Thunderbird\\Profiles\\isfi13ur.default\\datareporting',
  'Type': 'file'},
 'relation': 'EVENT_OPEN',
 'label': 'benign'}

In [7]:
count = 0
for event in tqdm(data):
    if event['relation'] != "NO_CORRESPONDING_RELATION":
        count += 1
        print(event)
        if count == 5:
            break

  0%|          | 0/42899811 [00:00<?, ?it/s]

{'src_uuid': '966D983C-607E-4039-BEEF-5E76BC0A4DA8', 'src_node': {'Cmdline': 'None', 'Type': 'process', 'Pid': 9432}, 'dst_uuid': '1987EE68-E918-4765-8082-3DE89890A305', 'dst_node': {'Filepath': '\\Device\\HarddiskVolume2\\Users\\admin\\AppData\\Roaming\\Thunderbird\\Profiles\\isfi13ur.default\\datareporting', 'Type': 'file'}, 'relation': 'EVENT_OPEN', 'label': 'benign'}
{'src_uuid': '966D983C-607E-4039-BEEF-5E76BC0A4DA8', 'src_node': {'Cmdline': 'None', 'Type': 'process', 'Pid': 9432}, 'dst_uuid': '1987EE68-E918-4765-8082-3DE89890A305', 'dst_node': {'Filepath': '\\Device\\HarddiskVolume2\\Users\\admin\\AppData\\Roaming\\Thunderbird\\Profiles\\isfi13ur.default\\datareporting', 'Type': 'file'}, 'relation': 'EVENT_OPEN', 'label': 'benign'}
{'src_uuid': '966D983C-607E-4039-BEEF-5E76BC0A4DA8', 'src_node': {'Cmdline': 'None', 'Type': 'process', 'Pid': 9432}, 'dst_uuid': 'A6BD7B35-CACA-4557-BE82-33B752E35C00', 'dst_node': {'Filepath': 'Device\\HarddiskVolume2\\Users\\admin\\AppData\\Roaming\

### From here we transform the relation based on the mapping.yml

In [8]:
import json
import yaml

def read_yaml(file_path):
    """Read a YAML file and return the parsed data."""
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

yaml_file_path = './data/darpa_synthesized_mapping.yml'

yaml_file = read_yaml(yaml_file_path)



In [9]:
print(type(yaml_file))
print(len(yaml_file))
# print(yaml_file)
yaml_file

<class 'dict'>
4


{'file': {'attr': {'filepath': 'File Name'},
  'relation': {'EVENT_CREATE_OBJECT': 'CreateFile',
   'EVENT_CHANGE_PRINCIPAL': 'SetBasicInformationFile',
   'EVENT_EXECUTE': 'Process Create',
   'EVENT_LOADLIBRARY': 'Load Image',
   'EVENT_MODIFY_FILE_ATTRIBUTES': 'SetBasicInformationFile',
   'EVENT_OPEN': 'CreateFile',
   'EVENT_READ': 'ReadFile',
   'EVENT_UNLINK': 'SetDispositionInformationFile',
   'EVENT_WRITE': 'WriteFile',
   'EVENT_CHECK_FILE_ATTRIBUTES': 'QueryBasicInformationFile',
   'EVENT_CLOSE': 'CloseFile',
   'EVENT_RENAME': 'SetRenameInformationFile',
   'EVENT_FCNTL': 'CreateFileMapping',
   'EVENT_OTHER': 'NO_CORRESPONDING_RELATION(EVENT_OTHER)'}},
 'network': {'attr': {'localAddress': 'Source Ip',
   'localPort': 'Source Port',
   'remoteAddress': 'Destination Ip',
   'remotePort': 'Destination Port'},
  'relation': {'EVENT_CONNECT': 'TCP Connect',
   'EVENT_RECVFROM': 'TCP Receive',
   'EVENT_SENDTO': 'TCP Send',
   'EVENT_ACCEPT': 'TCP Accept',
   'EVENT_BIND': 'N

In [10]:
def map_relation_to_synthesized(data, mapping):
    """Map relation in the data from DARPA format to synthesized format using the provided mapping."""
    for event in tqdm(data):
        src_type = event['src_node']['Type']
        dst_type = event['dst_node']['Type']
        darpa_relation = event['relation']

        # Check if the relation mapping exists for source type
        if darpa_relation in mapping[src_type]['relation']:
            event['relation'] = mapping[src_type]['relation'][darpa_relation]
        # If not found in source type, check in destination type
        elif darpa_relation in mapping[dst_type]['relation']:
            event['relation'] = mapping[dst_type]['relation'][darpa_relation]
        # If not found in both, set to 'NO_CORRESPONDING_RELATION'
        else:
            event['relation'] = "NO_CORRESPONDING_RELATION"
    return data

mapped_data = map_relation_to_synthesized(data, yaml_file)
# data_test

  0%|          | 0/42899811 [00:00<?, ?it/s]

In [11]:
count = 0
for event in tqdm(mapped_data):
    if event['relation'] == "NO_CORRESPONDING_RELATION":
        count += 1
        print(event)
        if count == 5:
            break

  0%|          | 0/42899811 [00:00<?, ?it/s]

{'src_uuid': '966D983C-607E-4039-BEEF-5E76BC0A4DA8', 'src_node': {'Cmdline': 'None', 'Type': 'process', 'Pid': 9432}, 'dst_uuid': '1987EE68-E918-4765-8082-3DE89890A305', 'dst_node': {'Filepath': '\\Device\\HarddiskVolume2\\Users\\admin\\AppData\\Roaming\\Thunderbird\\Profiles\\isfi13ur.default\\datareporting', 'Type': 'file'}, 'relation': 'Process Create', 'label': 'benign'}
{'src_uuid': '966D983C-607E-4039-BEEF-5E76BC0A4DA8', 'src_node': {'Cmdline': 'None', 'Type': 'process', 'Pid': 9432}, 'dst_uuid': '1987EE68-E918-4765-8082-3DE89890A305', 'dst_node': {'Filepath': '\\Device\\HarddiskVolume2\\Users\\admin\\AppData\\Roaming\\Thunderbird\\Profiles\\isfi13ur.default\\datareporting', 'Type': 'file'}, 'relation': 'Process Create', 'label': 'benign'}
{'src_uuid': '966D983C-607E-4039-BEEF-5E76BC0A4DA8', 'src_node': {'Cmdline': 'None', 'Type': 'process', 'Pid': 9432}, 'dst_uuid': 'A6BD7B35-CACA-4557-BE82-33B752E35C00', 'dst_node': {'Filepath': 'Device\\HarddiskVolume2\\Users\\admin\\AppData\\

#### Here we analyze the num of the label with "NO_CORRESPONDING_RELATION"

In [12]:
count == 0 
for event in tqdm(data):
    if event['relation'] == "NO_CORRESPONDING_RELATION":
        count += 1

print(count)

  0%|          | 0/42899811 [00:00<?, ?it/s]

1887495


In [13]:
count == 0 
for event in tqdm(mapped_data):
    if event['relation'] == "NO_CORRESPONDING_RELATION":
        count += 1

print(count)

  0%|          | 0/42899811 [00:00<?, ?it/s]

3774985


In [14]:
def print_first_five_events_with_no_corresponding_relation(original_data, mapped_data):
    count = 0
    for original_event, mapped_event in tqdm(zip(original_data, mapped_data)):
#         if mapped_event['relation'] == "NO_CORRESPONDING_RELATION":
        if mapped_event['relation'] == "NO_CORRESPONDING_RELATION" and mapped_event['label'] == "attack":
            print("Original Event:", original_event)
            print("Mapped Event:", mapped_event)
            print()  # Newline for separation
            count += 1
            if count == 5:
                break
                
print_first_five_events_with_no_corresponding_relation(data, mapped_data)

0it [00:00, ?it/s]

Original Event: {'src_uuid': '107A5E79-B50A-4D91-8480-D2EBE2E0F8C6', 'src_node': {'Cmdline': 'None', 'Type': 'process', 'Pid': 8560}, 'dst_uuid': '7082E8FD-A4A6-4198-82D7-1A3165474B89', 'dst_node': {'Cmdline': '"hJauWl01"', 'Type': 'process', 'Pid': 2760}, 'relation': 'NO_CORRESPONDING_RELATION', 'label': 'attack'}
Mapped Event: {'src_uuid': '107A5E79-B50A-4D91-8480-D2EBE2E0F8C6', 'src_node': {'Cmdline': 'None', 'Type': 'process', 'Pid': 8560}, 'dst_uuid': '7082E8FD-A4A6-4198-82D7-1A3165474B89', 'dst_node': {'Cmdline': '"hJauWl01"', 'Type': 'process', 'Pid': 2760}, 'relation': 'NO_CORRESPONDING_RELATION', 'label': 'attack'}

Original Event: {'src_uuid': '107A5E79-B50A-4D91-8480-D2EBE2E0F8C6', 'src_node': {'Cmdline': 'None', 'Type': 'process', 'Pid': 8560}, 'dst_uuid': '7082E8FD-A4A6-4198-82D7-1A3165474B89', 'dst_node': {'Cmdline': '"hJauWl01"', 'Type': 'process', 'Pid': 2760}, 'relation': 'NO_CORRESPONDING_RELATION', 'label': 'attack'}
Mapped Event: {'src_uuid': '107A5E79-B50A-4D91-848

In [15]:
keys = set()
for d in tqdm(data):
    if d['dst_node'].get('Type') == 'network':
        keys.update(d['dst_node'].keys())

print(keys)

  0%|          | 0/42899811 [00:00<?, ?it/s]

{'Ipprotocol', 'Localaddress', 'Localport', 'Remoteport', 'Remoteaddress', 'Type'}


In [19]:
keys = set()
for d in tqdm(mapped_data):
    if d['dst_node'].get('Type') == 'network':
        keys.update(d['dst_node'].keys())

print(keys)

  0%|          | 0/42899811 [00:00<?, ?it/s]

{'Ipprotocol', 'Localaddress', 'Localport', 'Remoteport', 'Remoteaddress', 'Type'}


In [16]:
keys = set()
for d in tqdm(data):
    if d['dst_node']['Type'] == 'process':
        keys = keys.union(d['dst_node'].keys())
keys

  0%|          | 0/42899811 [00:00<?, ?it/s]

{'Cmdline', 'Pid', 'Type'}

In [17]:
keys = set()
for d in tqdm(data):
    if d['dst_node']['Type'] == 'registry':
        keys = keys.union(d['dst_node'].keys())
keys

  0%|          | 0/42899811 [00:00<?, ?it/s]

{'Key', 'Type'}

In [18]:
keys = set()
for d in tqdm(data):
    if d['dst_node']['Type'] == 'file':
        keys = keys.union(d['dst_node'].keys())
keys

  0%|          | 0/42899811 [00:00<?, ?it/s]

{'Filepath', 'Type'}

In [20]:
save_data = list()
for d in tqdm(data):
    src_uuid = d['src_uuid']
    src_node = d['src_node']
    dst_uuid = d['dst_uuid']
    dst_node = d['dst_node']
    relation = d['relation']
    label = d['label']

    # 处理 src_node 中的 Cmdline
    src_cmdline = 'nan' if src_node['Cmdline'] == 'None' else src_node['Cmdline']

    # 初始化 dst_value
    dst_value = 'nan'

    new_row = {
        'src_uuid': src_uuid, 
        'src': f'process:{src_node["Pid"]}&{src_cmdline}',
        'dst_uuid': dst_uuid, 
        'dst': None, 
        'relation': relation, 
        'dst_type': dst_node['Type'].lower(),
        'label': label
    }

    # 根据 dst_node 的类型构造 dst 字段
    if new_row['dst_type'] == 'file':
        # 如果有 'Name' 字段，添加到 dst
        new_row['dst'] = new_row['dst_type'] + ':' + dst_node.get('Name', 'nan')
    elif new_row['dst_type'] == 'network':
        # 如果有网络相关字段，添加到 dst
        new_row['dst'] = new_row['dst_type'] + ':' + dst_node.get('Srcaddress', 'nan') + '&' + dst_node.get('Dstaddress', 'nan') + '&' + dst_node.get('Port', 'nan')
    elif new_row['dst_type'] == 'registry':
        # 使用 Key 字段
        new_row['dst'] = new_row['dst_type'] + ':' + dst_node['Key']
    elif new_row['dst_type'] == 'process':
        # 处理 dst_node 中的 Cmdline
        dst_cmdline = 'nan' if dst_node['Cmdline'] == 'None' else dst_node['Cmdline']
        new_row['dst'] = new_row['dst_type'] + ':' + f'{dst_node["Pid"]}&{dst_cmdline}'

    # 替换空格
    new_row['src'] = new_row['src'].replace(' ', '_')
    new_row['dst'] = new_row['dst'].replace(' ', '_')
    
    save_data.append(new_row)

df = pd.DataFrame(save_data)
print(df.shape)


  0%|          | 0/42899811 [00:00<?, ?it/s]

(42899811, 7)


In [21]:
df.head(10)

,src_uuid,src,dst_uuid,dst,relation,dst_type,label
0,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,1987EE68-E918-4765-8082-3DE89890A305,file:nan,Process Create,file,benign
1,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,1987EE68-E918-4765-8082-3DE89890A305,file:nan,Process Create,file,benign
2,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:nan,Process Create,file,benign
3,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:nan,Process Create,file,benign
4,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:nan,Process Create,file,benign
5,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:nan,CloseFile,file,benign
6,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:nan,CloseFile,file,benign
7,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:nan,CloseFile,file,benign
8,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,F17B89A1-F883-41C2-B0FD-F4FD5B7C21A2,file:nan,CreateFile,file,benign
9,509DFF9D-9792-49A5-8DCC-F60F29425F5E,process:6128&nan,084E9EFD-62AA-4FE4-B9DC-118DC1D0222F,file:nan,CreateFileMapping,file,benign


In [94]:
row_12000000 = df.iloc[12000000]
print(row_12000000)

src_uuid                 37E4A4E7-C628-4099-9058-D7D10B901512
src         process:8120&"C:\Program_Files\Mozilla_Firefox...
dst_uuid                 3F3FCFE5-BA84-4DB1-A538-4A2674E5511D
dst                                       network:nan&nan&nan
relation                                             TCP Send
dst_type                                              network
label                                                  benign
Name: 12000000, dtype: object


In [ ]:
data[0]

In [45]:
mapped_data[0]

{'src_uuid': '966D983C-607E-4039-BEEF-5E76BC0A4DA8',
 'src_node': {'Cmdline': 'None', 'Type': 'process', 'Pid': 9432},
 'dst_uuid': '1987EE68-E918-4765-8082-3DE89890A305',
 'dst_node': {'Filepath': '\\Device\\HarddiskVolume2\\Users\\admin\\AppData\\Roaming\\Thunderbird\\Profiles\\isfi13ur.default\\datareporting',
  'Type': 'file'},
 'relation': 'Process Create',
 'label': 'benign'}

In [43]:
data[1000000]

{'src_uuid': '6654BFF3-F769-43C6-B91A-15C087F1DBB5',
 'src_node': {'Cmdline': 'None', 'Type': 'process', 'Pid': 7780},
 'dst_uuid': '5AC4D736-CF60-40C6-9A13-AF49457A3574',
 'dst_node': {'Filepath': '\\Device\\HarddiskVolume2\\Windows\\System32\\Windows.CloudStore.dll',
  'Type': 'file'},
 'relation': 'QueryBasicInformationFile',
 'label': 'benign'}

In [79]:
mapped_data[1000000]

{'src_uuid': '6654BFF3-F769-43C6-B91A-15C087F1DBB5',
 'src_node': {'Cmdline': 'None', 'Type': 'process', 'Pid': 7780},
 'dst_uuid': '5AC4D736-CF60-40C6-9A13-AF49457A3574',
 'dst_node': {'Filepath': '\\Device\\HarddiskVolume2\\Windows\\System32\\Windows.CloudStore.dll',
  'Type': 'file'},
 'relation': 'NO_CORRESPONDING_RELATION',
 'label': 'benign'}

In [95]:
data[12000000]

{'src_uuid': '37E4A4E7-C628-4099-9058-D7D10B901512',
 'src_node': {'Cmdline': '"C:\\Program Files\\Mozilla Firefox\\firefox.exe" ',
  'Type': 'process',
  'Pid': 8120},
 'dst_uuid': '3F3FCFE5-BA84-4DB1-A538-4A2674E5511D',
 'dst_node': {'Localaddress': '127.0.0.1',
  'Localport': 61396,
  'Remoteaddress': '127.0.0.1',
  'Remoteport': 61395,
  'Ipprotocol': 6,
  'Type': 'network'},
 'relation': 'TCP Send',
 'label': 'benign'}

In [78]:
mapped_data[12000000]

{'src_uuid': '37E4A4E7-C628-4099-9058-D7D10B901512',
 'src_node': {'Cmdline': '"C:\\Program Files\\Mozilla Firefox\\firefox.exe" ',
  'Type': 'process',
  'Pid': 8120},
 'dst_uuid': '3F3FCFE5-BA84-4DB1-A538-4A2674E5511D',
 'dst_node': {'Localaddress': '127.0.0.1',
  'Localport': 61396,
  'Remoteaddress': '127.0.0.1',
  'Remoteport': 61395,
  'Ipprotocol': 6,
  'Type': 'network'},
 'relation': 'TCP Send',
 'label': 'benign'}

In [22]:
save_data = list()
for d in tqdm(mapped_data):
    src_uuid = d['src_uuid']
    src_node = d['src_node']
    dst_uuid = d['dst_uuid']
    dst_node = d['dst_node']
    relation = d['relation']
    label = d['label']

    # 处理 src_node 中的 Cmdline
    src_cmdline = 'nan' if src_node['Cmdline'] == 'None' else src_node['Cmdline']

    # 初始化 dst_value
    dst_value = 'nan'

    new_row = {
        'src_uuid': src_uuid, 
        'src': f'process:{src_node["Pid"]}&{src_cmdline}',
        'dst_uuid': dst_uuid, 
        'dst': None, 
        'relation': relation, 
        'dst_type': dst_node['Type'].lower(),
        'label': label
    }

    # 根据 dst_node 的类型构造 dst 字段
    if new_row['dst_type'] == 'file':
        # 如果有 'Name' 字段，添加到 dst
        new_row['dst'] = new_row['dst_type'] + ':' + dst_node.get('Name', 'nan')
    elif new_row['dst_type'] == 'network':
        # 如果有网络相关字段，添加到 dst
        new_row['dst'] = new_row['dst_type'] + ':' + dst_node.get('Srcaddress', 'nan') + '&' + dst_node.get('Dstaddress', 'nan') + '&' + dst_node.get('Port', 'nan')
    elif new_row['dst_type'] == 'registry':
        # 使用 Key 字段
        new_row['dst'] = new_row['dst_type'] + ':' + dst_node['Key']
    elif new_row['dst_type'] == 'process':
        # 处理 dst_node 中的 Cmdline
        dst_cmdline = 'nan' if dst_node['Cmdline'] == 'None' else dst_node['Cmdline']
        new_row['dst'] = new_row['dst_type'] + ':' + f'{dst_node["Pid"]}&{dst_cmdline}'

    # 替换空格 
    new_row['src'] = new_row['src'].replace(' ', '_')
    new_row['dst'] = new_row['dst'].replace(' ', '_')
    
    save_data.append(new_row)

df_mapped = pd.DataFrame(save_data)
print(df_mapped.shape)

  0%|          | 0/42899811 [00:00<?, ?it/s]

(42899811, 7)


In [93]:
# df_mapped.head(10)
row_12000000 = df_mapped.iloc[12000000]
print(row_12000000)

src_uuid                 37E4A4E7-C628-4099-9058-D7D10B901512
src         process:8120&"C:\Program_Files\Mozilla_Firefox...
dst_uuid                 3F3FCFE5-BA84-4DB1-A538-4A2674E5511D
dst                                       network:nan&nan&nan
relation                                             TCP Send
dst_type                                              network
label                                                  benign
Name: 12000000, dtype: object


In [89]:
save_data = list()

print(len(save_data))

0


In [23]:
df_mapped.head(10)

,src_uuid,src,dst_uuid,dst,relation,dst_type,label
0,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,1987EE68-E918-4765-8082-3DE89890A305,file:nan,Process Create,file,benign
1,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,1987EE68-E918-4765-8082-3DE89890A305,file:nan,Process Create,file,benign
2,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:nan,Process Create,file,benign
3,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:nan,Process Create,file,benign
4,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:nan,Process Create,file,benign
5,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:nan,CloseFile,file,benign
6,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:nan,CloseFile,file,benign
7,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:nan,CloseFile,file,benign
8,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&nan,F17B89A1-F883-41C2-B0FD-F4FD5B7C21A2,file:nan,CreateFile,file,benign
9,509DFF9D-9792-49A5-8DCC-F60F29425F5E,process:6128&nan,084E9EFD-62AA-4FE4-B9DC-118DC1D0222F,file:nan,CreateFileMapping,file,benign


## entity2id.txt, relation2id, entity2id_objType.txt, relation2id_type.txt

- entity set is the unique source  
- relation set is the unique relation

In [24]:
entity_set = set(df_mapped['src']).union(set(df['dst']))
relation_set = set(df_mapped['relation'])
print(len(entity_set))
print(len(relation_set))

26868
23


In [30]:
def save_entity2id(entity_set):
    entity_idx_dict = dict()
    msg = f'{len(entity_set)}\n'
    msg2 = f'{len(entity_set)}\n'
    obj_types = set()
    for idx, entity in enumerate(entity_set):
        tmp = entity.split(':')
        obj_type = tmp[0]
        entity = ':'.join(tmp[1:])
        obj_types.add(obj_type)
        entity = str(entity).replace(' ', '_')
#         if entity in entity_idx_dict:
#             print('new:', idx, entity, obj_type)
#             print('existed:', entity_idx_dict[entity], entity_obj_dict[entity])
        if entity in entity_idx_dict:
            print('new:', idx, entity, obj_type)
            print('existed:', entity_idx_dict[entity])
        msg += f'{entity}\t{idx}\n'
        msg2 += f'{obj_type}\t{idx}\n'
        entity_idx_dict[entity] = idx
#         entity_obj_dict[entity] = obj_type
    print(obj_types)

    with open(f'{OUTPUT_PATH}/entity2id.txt','w') as f:
        f.write(msg.strip())
    with open(f'{OUTPUT_PATH}/entity2id_objType.txt','w') as f:
        f.write(msg2.strip())
    print(f'Saved {OUTPUT_PATH}/entity2id.txt')
    print(f'Saved {OUTPUT_PATH}/entity2id_objType.txt')
    
    return entity_idx_dict


def save_relation2id(relation_set, yaml_mapping):
    relation_idx_dict = dict()
    with open(f'{OUTPUT_PATH}/relation2id.txt', 'w') as f, \
         open(f'{OUTPUT_PATH}/relation2id_type.txt', 'w') as f_type:
        
        msg = f'{len(relation_set)}\n'
        msg_type = f'{len(relation_set)}\n'

        # 创建一个反向映射，将合成的关系名称映射到其类型
        reverse_mapping = {}
        for object_type, relations in yaml_mapping.items():
            for darpa_rel, synth_rel in relations['relation'].items():
                # 将下划线替换为空格
                synth_rel = synth_rel.replace('_', ' ')
                reverse_mapping[synth_rel] = object_type

        for idx, relation in enumerate(relation_set):
            # 将下划线替换为空格
            relation = relation.replace('_', ' ')
            rel_type = reverse_mapping.get(relation, 'unknown')  # 如果找不到类型，则使用 'unknown'

            msg += f'{relation}\t{idx}\n'
            msg_type += f'{rel_type}\t{idx}\n'
            relation_idx_dict[relation] = idx

        f.write(msg.strip())
        f_type.write(msg_type.strip())

    print(f'Saved {OUTPUT_PATH}/relation2id.txt')
    print(f'Saved {OUTPUT_PATH}/relation2id_type.txt')
    return relation_idx_dict

entity_id_dict = save_entity2id(entity_set)
rel_id_dict = save_relation2id(relation_set, yaml_file)

{'process', 'registry', 'file', 'network'}
Saved ./data/3_openKE/synthesize/entity2id.txt
Saved ./data/3_openKE/synthesize/entity2id_objType.txt
Saved ./data/3_openKE/synthesize/relation2id.txt
Saved ./data/3_openKE/synthesize/relation2id_type.txt


In [26]:
print(len(entity_id_dict))

26868


## train2id.txt, valid2id.txt, test2id.txt, train2id_labels.txt, valid2id_labels.txt, test2id_labels.txt

In [32]:
triplets = list(zip(df_mapped['src'],df_mapped['dst'],df_mapped['relation'],df_mapped['dst_type'],df_mapped['label']))
print(len(triplets))

for t in tqdm(triplets[:30]):
    print(t[4], end=',')

42899811


  0%|          | 0/30 [00:00<?, ?it/s]

benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,benign,

In [35]:
new_triplets = list()
for triplet in tqdm(triplets):
    src_entity = str(':'.join(triplet[0].split(':')[1:])).replace(' ', '_')
    dst_entity = str(':'.join(triplet[1].split(':')[1:])).replace(' ', '_')

    # 使用空格而不是下划线来匹配 rel_id_dict
    relation = triplet[2].replace('_', ' ')

    new_triplets.append((
        entity_id_dict[src_entity],
        entity_id_dict[dst_entity],
        rel_id_dict[relation],  # 使用修改后的关系名称
        triplet[3],
        triplet[4]
    ))

len(new_triplets)


  0%|          | 0/42899811 [00:00<?, ?it/s]

42899811

In [38]:
print(new_triplets[:5])

[(7042, 8575, 1, 'file', 'benign'), (7042, 8575, 1, 'file', 'benign'), (7042, 8575, 1, 'file', 'benign'), (7042, 8575, 1, 'file', 'benign'), (7042, 8575, 1, 'file', 'benign')]


In [33]:
from collections import Counter
Counter(df_mapped['label']).most_common()

[('benign', 42898977), ('attack', 834)]

In [39]:
# new_triplets = list()
# for triplet in tqdm(triplets):
#     src_entity = str(':'.join(triplet[0].split(':')[1:])).replace(' ', '_')
#     dst_entity = str(':'.join(triplet[1].split(':')[1:])).replace(' ', '_')
#     new_triplets.append((
#         entity_id_dict[src_entity],
#         entity_id_dict[dst_entity],
#         rel_id_dict[triplet[2].replace(' ', '_')],
#         triplet[3],
#         triplet[4]
#     ))
# len(new_triplets)

In [40]:
TRAIN_SIZE = 1
VALID_SIZE = 0.1
TEST_SIZE = 0.1
train_dataset = np.array(new_triplets)
_, tmp = train_test_split(train_dataset, test_size=VALID_SIZE+TEST_SIZE, shuffle=True)
valid_dataset, test_dataset = train_test_split(tmp, test_size=TEST_SIZE/(VALID_SIZE+TEST_SIZE),shuffle=True)
print(f'train_dataset.shape: {train_dataset.shape} ({TRAIN_SIZE})')
print(f'valid_dataset.shape: {valid_dataset.shape} ({VALID_SIZE})')
print(f'test_dataset.shape: {test_dataset.shape} ({TEST_SIZE})')

train_dataset.shape: (42899811, 5) (1)
valid_dataset.shape: (4289981, 5) (0.1)
test_dataset.shape: (4289982, 5) (0.1)


In [41]:
def save_2id_txt(data, ent_fname, ent_label_fname):
    msg = f'{len(data)}\n'
    msg2 = f'{len(data)}\n'
    for ids in data:
        msg += f'{ids[0]} {ids[1]} {ids[2]}\n'
        msg2 += f'{ids[3]} {ids[4]}\n'
    with open(f'{OUTPUT_PATH}/{ent_fname}', 'w') as f:
        f.write(msg.strip())
    print(f'Saved {OUTPUT_PATH}/{ent_fname}')
    with open(f'{OUTPUT_PATH}/{ent_label_fname}', 'w') as f:
        f.write(msg2.strip())
    print(f'Saved {OUTPUT_PATH}/{ent_label_fname}')

save_2id_txt(train_dataset, 'train2id.txt', 'train2id_label.txt')
save_2id_txt(valid_dataset, 'valid2id.txt', 'valid2id_label.txt')
save_2id_txt(test_dataset, 'test2id.txt', 'test2id_label.txt')

Saved ./data/3_openKE/synthesize/train2id.txt
Saved ./data/3_openKE/synthesize/train2id_label.txt
Saved ./data/3_openKE/synthesize/valid2id.txt
Saved ./data/3_openKE/synthesize/valid2id_label.txt
Saved ./data/3_openKE/synthesize/test2id.txt
Saved ./data/3_openKE/synthesize/test2id_label.txt
